In [1]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, StringIndexer
from pyspark.sql.functions import length, size
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
import nltk

--------------------------------------------------------------------------- 
 ImportError Traceback (most recent call last)
 <command-4400432230767252> in <module> () 
 3 from pyspark . sql import SQLContext 
 4 sqlContext = SQLContext ( sc ) 
 ----> 5 import nltk 

 ImportError : No module named 'nltk'

In [2]:
##dataframe = sqlContext.read.format('csv').options(header='true', inferScheme='true').load('C:/Users/jazmi/OneDrive/Documents/DataMate/db/all.csv')
df = spark.table("all_csv")

df.columns

Out[ 5 ]: ['_c0', 'username', 'Gender', 'Education', 'Ethnicity', 'about_me_text_clean']

In [3]:
df = df.select('username', 'about_me_text_clean', 'Gender').na.drop()
df_length = df.withColumn('length', length(df['about_me_text_clean']))
df_length.show()

+--------------+--------------------+------+------+
 username| about_me_text_clean|Gender|length|
+--------------+--------------------+------+------+
 ROXYCHIC549| I am not an upgr...| F| 1615|
 tequilagirl69|Laid back lovable...| F| 175|
atxredneckgirl|Tx country music ...| F| 259|
 mel12th|I am here to look...| F| 338|
 Andi010274|Cant get into my ...| F| 47|
 nodutch|Music business go...| F| 67|
 lauranmr|Of all the dating...| F| 708|
 smbd75|Likes fun adventu...| F| 66|
 candipie1989|Single black fema...| F| 376|
 tif1985|Im on here just l...| F| 300|
 De1964|Please local only...| F| 1088|
 Trini1971|I am a Mom to a 1...| F| 564|
 FiaFia1213|I like to think I...| F| 341|
 SweetJojos| Up for anything| F| 15|
 Tawdra83|I’m a artist I co...| F| 423|
 LadyTexas254|not looking for a...| F| 233|
 Vigilia79|Hi there Im Chris...| F| 607|
 ayoung292|Music is life and...| F| 102|
 Foreverfungrl| Ask me Are| F| 10|
 southstar5|Single have two k...| F| 39|
+--------------+--------------------+------+------+
only showing top 20 rows

In [4]:
male_female_to_num = StringIndexer(inputCol='Gender', outputCol='label')

tokened_df = Tokenizer(inputCol="about_me_text_clean", outputCol="token_text")

stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')

hashing = HashingTF(inputCol='token_text', outputCol="hashedValues")

idf = IDF(minDocFreq = 10, inputCol="hashedValues", outputCol="idf_features")


In [5]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_features', 'length'], outputCol='features')

In [6]:
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[male_female_to_num, tokened_df, stop_remove, hashing, idf, clean_up])

In [7]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df_length)
cleaned = cleaner.transform(df_length)

In [8]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(262145,[4200,538...|
 0.0|(262145,[4333,844...|
 0.0|(262145,[9639,286...|
 0.0|(262145,[14,9639,...|
 0.0|(262145,[24417,37...|
 0.0|(262145,[9639,572...|
 0.0|(262145,[1998,961...|
 0.0|(262145,[8443,199...|
 0.0|(262145,[8449,134...|
 0.0|(262145,[4200,576...|
 0.0|(262145,[3326,408...|
 0.0|(262145,[9639,129...|
 0.0|(262145,[329,9639...|
 0.0|(262145,[16332,10...|
 0.0|(262145,[2437,963...|
 0.0|(262145,[9988,133...|
 0.0|(262145,[4200,963...|
 0.0|(262145,[15889,24...|
 0.0|(262145,[109810,1...|
 0.0|(262145,[15664,23...|
+-----+--------------------+
only showing top 20 rows

In [9]:
from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [10]:
test_results = predictor.transform(testing)
test_results.show(5)

+----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 username| about_me_text_clean|Gender|length|label| token_text| stop_tokens| hashedValues| idf_features| features| rawPrediction| probability|prediction|
+----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 15winstom|Iam a traveler I ...| M| 137| 1.0|[iam, a, traveler...|[iam, traveler, a...|(262144,[4788,609...|(262144,[4788,609...|(262145,[11724,24...|[-596.54913875400...|[0.99999999998621...| 0.0|
 19edt70|Music snob love l...| M| 26| 1.0|[music, snob, lov...|[music, snob, lov...|(262144,[33053,63...|(262144,[33053,63...|(262145,[33053,63...|[-69.046188189177...|[0.98566843209601...| 0.0|
 1KindOfLove|Hi my name is She...| F| 854| 0.0|[hi, my, name, is...|[hi, name, sheila...|(262144,[9639,144...|(262144,[9639,144...|(262145,[9639,144...|[-3274.6976735079...|[1.0,2.3281027418...| 0.0|
1PrettyFeetChef1|BBW Entrepreneur...| F| 336| 0.0|[bbw, , entrepren...|[bbw, , entrepren...|(262144,[9639,163...|(262144,[9639,163...|(262145,[9639,163...|[-1089.4348342104...|[1.0,5.8482483106...| 0.0|
 210Frances|Hello My name is ...| F| 229| 0.0|[hello, my, name,...|[hello, name, fra...|(262144,[8443,158...|(262144,[8443,158...|(262145,[8443,158...|[-825.61464493235...|[1.0,6.8045269544...| 0.0|
+----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.375637